In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()

driver.get("https://www.ulaval.ca/etudes/programmes")


title = driver.title
print(title)

driver.implicitly_wait(0.5)

results_element= driver.find_element(By.ID,"resultats")
print(results_element.value)
# text_box = driver.find_element(by=By.NAME, value="my-text")
# submit_button = driver.find_element(by=By.CSS_SELECTOR, value="button")

# text_box.send_keys("Selenium")
# submit_button.click()

# message = driver.find_element(by=By.ID, value="message")
# text = message.text

# driver.quit()

Trouvez votre programme d'études | Université Laval


AttributeError: 'WebElement' object has no attribute 'value'